### 参考
1. [Spark_SQL_cheatsheet_code.ipynb](https://github.com/ShowMeAI-Hub/awesome-AI-cheatsheets/blob/main/Spark/Spark_SQL_cheatsheet_code.ipynb)， 镜像在gitee [ShowMeAI-Hub--awesome-AI-cheatsheets.git](https://gitee.com/mirrr/ShowMeAI-Hub--awesome-AI-cheatsheets/tree/main/Spark)

### 环境安装

https://cdn.azul.com/zulu/bin/zulu8.76.0.17-ca-jdk8.0.402-linux_x64.tar.gz

https://archive.apache.org/dist/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz

In [1]:
%%bash
date && pwd

which python
which pip

file /app/zulu8.76.0.17-ca-jdk8.0.402-linux_x64 /app/spark-2.4.8-bin-hadoop2.7
md5sum /app/pack/spark-2.4.8-bin-hadoop2.7.tgz /app/pack/zulu8.76.0.17-ca-jdk8.0.402-linux_x64.tar.gz

ls -lh /fridaAnlzAp/frida_js/*RunBuszJs*.log

pip install findspark

2024年 03月 25日 星期一 20:06:38 CST
/fridaAnlzAp/analyze_by_spark_GraphFrame
/app/Miniconda3-py37_4.12.0/bin/python
/app/Miniconda3-py37_4.12.0/bin/pip
/app/zulu8.76.0.17-ca-jdk8.0.402-linux_x64: directory
/app/spark-2.4.8-bin-hadoop2.7:             directory
3af9ae26e91360c0f39a5647d9279715  /app/pack/spark-2.4.8-bin-hadoop2.7.tgz
3d8073a1e7bc71a0c53bbbbad590dad2  /app/pack/zulu8.76.0.17-ca-jdk8.0.402-linux_x64.tar.gz
-rwxrwxrwx 1 z z 263M  3月 24 16:57 /fridaAnlzAp/frida_js/frida-trace-out-RunBuszJs-1711270533.log
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/a4/cb/7d2bb508f4ca00a043fd53e8156c11767799d3f534bf451a0942211d5def/findspark-2.0.1-py2.py3-none-any.whl (4.4 kB)


In [1]:
import os
os.environ["JAVA_HOME"]="/app/zulu8.76.0.17-ca-jdk8.0.402-linux_x64"
# os.environ["SPARK_HOME"]="/app/spark-2.4.8-bin-hadoop2.7"



In [2]:
import findspark
findspark.init(spark_home="/app/spark-2.4.8-bin-hadoop2.7")

In [27]:
findspark.init?

Signature:
findspark.init(
    spark_home=None,
    python_path=None,
    edit_rc=False,
    edit_profile=False,
)
Docstring:
Make pyspark importable.

Sets environment variables and adds dependencies to sys.path.
If no Spark location is provided, will try to find an installation.

Parameters
----------
spark_home : str, optional, default = None
    Path to Spark installation, will try to find automatically
    if not provided.
python_path : str, optional, default = None
    Path to Python for Spark workers (PYSPARK_PYTHON),
    will use the currently running Python if not provided.
edit_rc : bool, optional, default = False
    Whether to attempt to persist changes by appending to shell
    config.
edit_profile : bool, optional, default = False
    Whether to create an IPython startup file to automatically
    configure and import pyspark.
File:      /app/Miniconda3-py310_22.11.1-1/lib/python3.10/site-packages/findspark.py
Type:      function

In [3]:
from pyspark.sql import SparkSession
#必须用py3.7 , 否则 这一行报错 TypeError: 'bytes' object cannot be interpreted as an integer

SparkSession 用于创建数据帧，将数据帧注册为表，执行 SQL 查询，缓存表及读取 Parquet 文件。

In [4]:
spark = SparkSession \
       .builder \
       .appName("analyze_by_spark_GraphFrame") \
       .config("spark.some.config.option", "some-value") \
       .getOrCreate()

24/03/25 20:16:18 WARN Utils: Your hostname, mchr resolves to a loopback address: 127.0.1.1; using 10.0.4.23 instead (on interface wlo1)
24/03/25 20:16:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/25 20:16:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
from pyspark.sql.types import *
from pyspark.sql import *
from pyspark.sql.functions import *

In [6]:
sc = spark.sparkContext

In [7]:
!wget https://github.com/awesome-AI-cheatsheets/tree/main/Spark/data/people.txt

--2024-03-25 20:16:51--  https://github.com/awesome-AI-cheatsheets/tree/main/Spark/data/people.txt
正在解析主机 github.com (github.com)... 10.0.4.23
正在连接 github.com (github.com)|10.0.4.23|:443... 失败：连接被拒绝。
